# Digital Twin Supply Chain Simulation — Answer Key

In [ ]:
import simpy, random, numpy as np, pandas as pd
random.seed(42)

class Pipe:
    def __init__(self, env): self.env=env; self.store=simpy.Store(env)
    def put(self, value): return self.store.put((self.env.now, value))
    def get(self): return self.store.get()

def factory(env, out_pipe, mean_prod=100, std_prod=10):
    while True:
        yield env.timeout(1); qty=max(0,int(random.gauss(mean_prod,std_prod))); out_pipe.put(qty)

def dc(env, in_pipe, out_pipe, capacity=300, logs=[]):
    on_hand=0
    while True:
        t, inc = yield in_pipe.get()
        on_hand += inc
        ship = min(on_hand, capacity)
        on_hand -= ship
        logs.append((t,"dc",on_hand,ship))
        yield env.timeout(1); out_pipe.put(ship)

def store(env, in_pipe, demand_mean=80, logs=[]):
    on_hand=0
    while True:
        t, inc = yield in_pipe.get()
        on_hand += inc
        d = np.random.poisson(demand_mean)
        filled = min(on_hand, d); on_hand -= filled
        logs.append((t,"store",on_hand,filled))
        yield env.timeout(1)

env = simpy.Environment()
f2d = Pipe(env); d2s = Pipe(env); logs=[]
env.process(factory(env, f2d))
env.process(dc(env, f2d, d2s, logs=logs))
env.process(store(env, d2s, logs=logs))
env.run(until=300)
df = pd.DataFrame(logs, columns=["time","node","on_hand","flow"])
display(df.head())
df.to_csv("../reports/sim_logs.csv", index=False)
print("Simulation complete; logs written to reports/sim_logs.csv")
